In [1]:
import pandas as pd
import psycopg2
import yfinance as yf

# CONNECT to REDSHIFT

In [3]:
with open(r"C:\Users\juanm\creds\pwd_redshift.txt",'r') as f:
    pwd= f.read()

connection_params = dict(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
        , dbname='data-engineer-database'
        , user='juanmlacasa_coderhouse'
        , password=pwd
        , port='5439'
)

try:
    conn = psycopg2.connect(
        **connection_params
    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


# EXTRACT

In [4]:
# Setup the parameters for the data to be fetched
ticker_info = dict(ticker_symbol = 'AAPL', data_source='Yahoo Finance', source_type = 'Platform', interval='1d', start_date = '2023-01-01', end_date = None)


In [5]:

# Fetch data for Apple Inc. with a daily interval
data = yf.download(ticker_info['ticker_symbol'], interval=ticker_info['interval'], start=ticker_info['start_date'], end=ticker_info['end_date'])

# Display the data
print(data.head())

[*********************100%%**********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-01-03  130.279999  130.899994  124.169998  125.070000  124.538666   
2023-01-04  126.889999  128.660004  125.080002  126.360001  125.823189   
2023-01-05  127.129997  127.769997  124.760002  125.019997  124.488876   
2023-01-06  126.010002  130.289993  124.889999  129.619995  129.069321   
2023-01-09  130.470001  133.410004  129.889999  130.149994  129.597061   

               Volume  
Date                   
2023-01-03  112117500  
2023-01-04   89113600  
2023-01-05   80962700  
2023-01-06   87754700  
2023-01-09   70790800  


# TRANSFORM

In [13]:
def get_or_create_record(conn, search_value, table_name, entity):
    # Check if data source exists
    with conn.cursor() as cur:
        cur.execute(f"""
            SELECT {entity}_id FROM {table_name} WHERE {entity}_name = '{search_value}';
            """)
        try:
            record_id = cur.fetchone()[0]
        except TypeError:
            record_id = None
    if not record_id:
        # Insert new data source
        with conn.cursor() as cur:
            cur.execute(f"""
                INSERT INTO {table_name} ({entity}_name) VALUES ('{search_value}');
            """)
            cur.execute(f"""
                SELECT MAX({entity}_id) FROM {table_name}
            """)
            record_id = cur.fetchone()[0]
            conn.commit()
    return record_id

In [14]:
asset_id = get_or_create_record(conn, ticker_info['ticker_symbol'], "assets", "asset")
data_source_id = get_or_create_record(conn, ticker_info['data_source'], "data_sources", "source")
source_type_id = get_or_create_record(conn, ticker_info['source_type'], "source_types", "type")
print(asset_id, source_type_id, data_source_id)

4 2 3


# LOAD

In [15]:
data.reset_index(inplace=True)
data.columns = [c.lower().replace(' ', '_') for c in data.columns]
data.rename(columns={'date':'timestamp'}, inplace=True)
data['asset_id'] = asset_id
data['source_id'] = data_source_id

In [16]:
from psycopg2.extras import execute_values

def load_redshift(conn, table_name, dataframe):
    with conn.cursor() as curs:
        values = [tuple(x) for x in dataframe.to_numpy()]
        cols = '"'+'''", "'''.join(data.columns)+'"'
        # define INSERT INTO statement
        insert_sql = f"INSERT INTO {table_name} ({cols}) VALUES %s"
        # Execute the transaction to insert the data
        curs.execute("BEGIN")
        execute_values(curs, insert_sql, values)
        curs.execute("COMMIT")
    print('Proceso terminado')

In [17]:
load_redshift(conn, 'ohlcv_data', data)

Proceso terminado


In [94]:
# with conn.cursor() as curs:
#     curs.execute("ROLLBACK")
#     conn.commit()